In [1]:
"""
Created on Tue Nov 05 18:14 2024

Determine a zmin and zmax for the open cavity points to prescribe the patterns

@author: Clara Burgard
"""

'\nCreated on Tue Nov 05 18:14 2024\n\nDetermine a zmin and zmax for the open cavity points to prescribe the patterns\n\n@author: Clara Burgard\n'

In [2]:
import xarray as xr
import numpy as np
from multimelt.constants import *

In [3]:
%matplotlib qt5

QStandardPaths: error creating runtime directory '/run/user/2784' (Permission denied)


In [4]:
inputpath_raw = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/raw/'
inputpath_interim = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/interim/'
outputpath_interim = '/data/cburgard/PREPARE_FORCING/PREPARE_PRESCRIBED_MELT/interim/'

In [5]:
domain_cfg = xr.open_dataset(inputpath_raw + 'eORCA1.4.3_OpenSeas_OpenAllCav_ModStraights/eORCA1.4.3_OpenSeas_OpenAllCav_ModStraights_domain_cfg.nc')
file_mask_eORCA1 = xr.open_dataset(inputpath_interim + 'masks_for_eORCA1.nc').squeeze()
file_Justine = xr.open_dataset(inputpath_interim + 'Mask_Iceshelf_IMBIE2_v2_with_zmin_zmax_isfconc.nc').drop('quantile')

In [6]:
ID_open_list = [21,66,67,117,124,125,127,128]

In [8]:
file_mask_eORCA1

<xarray.Dataset>
Dimensions:          (y: 331, x: 360)
Coordinates:
    lat              (y, x) float64 ...
    lon              (y, x) float64 ...
Dimensions without coordinates: y, x
Data variables:
    mask_isf         (y, x) float32 ...
    floating_frac    (y, x) float64 ...
    mask_isf_open    (y, x) float32 ...
    mask_isf_closed  (y, x) float32 ...
    z_min            (y, x) float32 ...
    z_max            (y, x) float32 ...
    melt_isf_closed  (y, x) float64 ...
Attributes:
    Source:         Based on IMBIE2 (Mask_Iceshelf_IMBIE2_v2.nc), melt from D...
    Compatibility:  The masked variables are compatible with eORCA1.4.3_OpenS...
    Creator:        C. Burgard and P. Mathiot (Nov-05-2024)

In [7]:
isf_draft = domain_cfg['isf_draft'].where(np.isfinite(file_mask_eORCA1['mask_isf_open']))
z_min = isf_draft
z_max = isf_draft + 5

In [8]:
file_mask_eORCA1['z_min'] = file_mask_eORCA1['z_min'].combine_first(z_min)
file_mask_eORCA1['z_max'] = file_mask_eORCA1['z_max'].combine_first(z_max)

In [9]:
melt_Davison_file = xr.open_dataset(outputpath_interim + 'Davison23_basal_melt_pattern_on_eORCA1.nc').squeeze().drop('band') #['basal_melt']

In [10]:
melt_Davison_file

<xarray.Dataset>
Dimensions:     (y: 331, x: 360, nvertex: 4)
Coordinates:
    lon         (y, x) float64 ...
    lat         (y, x) float64 ...
Dimensions without coordinates: y, x, nvertex
Data variables:
    lon_bnds    (y, x, nvertex) float64 ...
    lat_bnds    (y, x, nvertex) float64 ...
    basal_melt  (y, x) float32 ...
    longitude   (y, x) float64 ...
    latitude    (y, x) float64 ...
Attributes:
    CDI:          Climate Data Interface version 2.1.0 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    history:      Wed Nov 06 18:15:36 2024: cdo -O -s -remapbil,/data/cburgar...
    CDO:          Climate Data Operators version 2.1.0 (https://mpimet.mpg.de...

In [37]:
melt_Gt = melt_Davison_file['basal_melt'] * domain_cfg['e1t'] * domain_cfg['e2t'] * rho_i * 10**(-12)

In [50]:
file_mask_eORCA1['melt_isf_open'] = melt_Gt.where(np.isfinite(file_mask_eORCA1['mask_isf_open'])).squeeze()

In [52]:
file_mask_eORCA1['melt_isf'] = file_mask_eORCA1['melt_isf_open'].combine_first(file_mask_eORCA1['melt_isf_closed'])

In [54]:
file_mask_eORCA1['melt_isf'].plot()

In [55]:
file_mask_eORCA1.to_netcdf(outputpath_interim + 'masks_for_eORCA1_prescribedmeltinopencav.nc')

In [43]:
file_mask_eORCA1['melt_isf_closed'].plot()

In [48]:
(file_mask_eORCA1['melt_isf_open'] == 0).plot()